<a href="https://colab.research.google.com/github/UlrikeDetective/code/blob/main/ADK_Learning_tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 Welcome to Your ADK Adventure - Tools & Memory! 🚀

Welcome, Agent Architect! This notebook is your guide to giving your AI agents two essential superpowers: custom tools and conversational memory.

By the end of this adventure, you will be able to:

- **Build a Foundational Agent**: Create a simple but effective AI agent from scratch using the Google Agent Development Kit (ADK).

- **Grant New Skills with Custom Tools**: Teach an agent to perform new tasks by connecting it to external APIs, like a real-time weather service.

- **Create a Team of Agents**: Assemble a multi-agent system where a primary agent can delegate specialized tasks to other agents.

- **Master Conversational Memory**: Understand the critical role of Sessions in enabling agents to remember previous interactions, handle feedback, and carry on a coherent conversation.


Let's get this adventure started!

## Author

HI, I'm Qingyue (Annie) Wang, a developer advocate and AI engineer at **Google**, passionate about helping developers build with AI and cloud technologies :)


If you have questions with this notebook, contact me on [LinkedIn](https://www.linkedin.com/in/qingyuewang/) , [X](https://twitter.com/qingyuewang) or email anniewangtech0510@Gmail.com


```
  (\__/)
  (•ㅅ•)
  /づ  📚      Enjoy learning AI Agents :)
```


-------------
### 🎁 🛑 Important Prerequisite: Setup Your Environment! 🛑 🎁
-----------------------------------------------------------------------------

👉 **Get Your API Key HERE**: https://codelabs.developers.google.com/onramp/instructions#1

 -----------------------------------------------------------------------------

```
 ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️
   /\_/\     /\_/\     /\_/\      /\_/\       /\_/\
  ( ^_^ )   ( -.- )   ( >_< )   ( =^.^= )    ( o_o )             
```


## Part 0: Setup & Authentication 🔑

First things first, let's get all our tools ready. This step installs the necessary libraries and securely configures your Google API key so your agents can access the power of Gemini.

In [1]:
!pip install google-adk google-generativeai -q

# --- Import all necessary libraries for our entire adventure ---
import os
import re
import asyncio
from IPython.display import display, Markdown
import google.generativeai as genai
from google.adk.agents import Agent
from google.adk.tools import google_search
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService, Session
from google.genai.types import Content, Part
from getpass import getpass

print("✅ All libraries are ready to go!")

✅ All libraries are ready to go!


/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "config_type" in "SequentialAgent" shadows an attribute in parent "BaseAgent"
  warnings.warn(


In [3]:
# --- Securely Configure Your API Key ---

# Prompt the user for their API key securely
api_key = getpass('Enter your Google API Key:')

# Get Your API Key HERE 👉 https://codelabs.developers.google.com/onramp/instructions#0
# Configure the generative AI library with the provided key
genai.configure(api_key=api_key)

# Set the API key as an environment variable for ADK to use
os.environ['GOOGLE_API_KEY'] = api_key

print("✅ API Key configured successfully! Let the fun begin.")

Enter your Google API Key:··········
✅ API Key configured successfully! Let the fun begin.


---
## Part 1: Your First Agent - The Day Trip Genie 🧞

Meet your first creation! The `day_trip_agent` is a simple but powerful assistant. We're making it a little smarter by teaching it to understand **budget constraints**.

* **Agent**: The brain of the operation, defined by its instructions, tools, and the AI model it uses.
* **Session**: The conversation history. For this simple agent, it's just a container for a single request-response.
* **Runner**: The engine that connects the `Agent` and the `Session` to process your request and get a response.

```
+--------------------------------------------------+
|         Spontaneous Day Trip Agent 🤖            |
|--------------------------------------------------|
|  Model: gemini-2.5-flash                         |
|  Description:                                    |
|   Generates full-day trip itineraries based on   |
|   mood, interests, and budget                    |
|--------------------------------------------------|
|  🔧 Tools:                                       |
|   - Google Search                                |
|--------------------------------------------------|
|  🧠 Capabilities:                                |
|   - Budget Awareness (cheap / splurge)           |
|   - Mood Matching (adventurous, relaxing, etc.)  |
|   - Real-Time Info (hours, events)               |
|   - Morning / Afternoon / Evening plan           |
+--------------------------------------------------+

            ▲
            |
    +------------------+
    |   User Input     |
    |------------------|
    |  Mood            |
    |  Interests       |
    |  Budget          |
    +------------------+

            |
            ▼

+--------------------------------------------------+
|             Output: Markdown Itinerary           |
|--------------------------------------------------|
| - Time blocks (Morning / Afternoon / Evening)    |
| - Venue names with links and hours               |
| - Budget-matching activities                     |
+--------------------------------------------------+
```


In [4]:
# --- Agent Definition ---

def create_day_trip_agent():
    """Create the Spontaneous Day Trip Generator agent"""
    return Agent(
        name="day_trip_agent",
        model="gemini-2.5-flash",
        description="Agent specialized in generating spontaneous full-day itineraries based on mood, interests, and budget.",
        instruction="""
        You are the "Spontaneous Day Trip" Generator 🚗 - a specialized AI assistant that creates engaging full-day itineraries.

        Your Mission:
        Transform a simple mood or interest into a complete day-trip adventure with real-time details, while respecting a budget.

        Guidelines:
        1. **Budget-Aware**: Pay close attention to budget hints like 'cheap', 'affordable', or 'splurge'. Use Google Search to find activities (free museums, parks, paid attractions) that match the user's budget.
        2. **Full-Day Structure**: Create morning, afternoon, and evening activities.
        3. **Real-Time Focus**: Search for current operating hours and special events.
        4. **Mood Matching**: Align suggestions with the requested mood (adventurous, relaxing, artsy, etc.).

        RETURN itinerary in MARKDOWN FORMAT with clear time blocks and specific venue names.
        """,
        tools=[google_search]
    )

day_trip_agent = create_day_trip_agent()
print(f"🧞 Agent '{day_trip_agent.name}' is created and ready for adventure!")

🧞 Agent 'day_trip_agent' is created and ready for adventure!


In [6]:
# --- A Helper Function to Run Our Agents ---
# We'll use this function throughout the notebook to make running queries easy.

async def run_agent_query(agent: Agent, query: str, session: Session, user_id: str, is_router: bool = False):
    """Initializes a runner and executes a query for a given agent and session."""
    print(f"\n🚀 Running query for agent: '{agent.name}' in session: '{session.id}'...")

    runner = Runner(
        agent=agent,
        session_service=session_service,
        app_name=agent.name
    )

    final_response = ""
    try:
        async for event in runner.run_async(
            user_id=user_id,
            session_id=session.id,
            new_message=Content(parts=[Part(text=query)], role="user")
        ):
            if not is_router:
                # Let's see what the agent is thinking!
                print(f"EVENT: {event}")
            if event.is_final_response():
                final_response = event.content.parts[0].text
    except Exception as e:
        final_response = f"An error occurred: {e}"

    if not is_router:
     print("\n" + "-"*50)
     print("✅ Final Response:")
     display(Markdown(final_response))
     print("-"*50 + "\n")

    return final_response

# --- Initialize our Session Service ---
# This one service will manage all the different sessions in our notebook.
session_service = InMemorySessionService()
my_user_id = "adk_adventurer_001"

In [8]:
# --- Let's test the Day Trip Genie! ---

async def run_day_trip_genie():
    # Create a new, single-use session for this query
    day_trip_session = await session_service.create_session(
        app_name=day_trip_agent.name,
        user_id=my_user_id
    )

    # Note the new budget constraint in the query!
    query = "Plan a busy and artsy (including street art and Art in architecture) day trip near Sunnyvale, CA. Keep it affordable!"
    print(f"🗣️ User Query: '{query}'")

    await run_agent_query(day_trip_agent, query, day_trip_session, my_user_id)

await run_day_trip_genie()

🗣️ User Query: 'Plan a busy and artsy (including street art and Art in architecture) day trip near Sunnyvale, CA. Keep it affordable!'

🚀 Running query for agent: 'day_trip_agent' in session: 'f23c5aaf-4863-4bb5-b622-0b25ca2c9b93'...
EVENT: content=Content(
  parts=[
    Part(
      text="""Get ready for a bustling and artistic adventure in Downtown San Jose, focusing on captivating street art and impressive architectural designs, all while keeping your budget in mind! San Jose, just a short drive from Sunnyvale, offers a rich tapestry of visual culture waiting to be explored.

Here's your affordable and artsy day trip itinerary:

---

## 🎨 Spontaneous Day Trip: Artsy San Jose on a Budget 🖼️

**Destination:** Downtown San Jose, CA (approx. 20-30 min drive from Sunnyvale)
**Mood:** Busy, Artsy (Street Art & Architecture)
**Budget:** Affordable (Free activities, budget-friendly food)

---

### ☀️ Morning (Architecture & Public Art Immersion)

*   **9:30 AM - 10:30 AM: Modern Masterpiece 

Get ready for a bustling and artistic adventure in Downtown San Jose, focusing on captivating street art and impressive architectural designs, all while keeping your budget in mind! San Jose, just a short drive from Sunnyvale, offers a rich tapestry of visual culture waiting to be explored.

Here's your affordable and artsy day trip itinerary:

---

## 🎨 Spontaneous Day Trip: Artsy San Jose on a Budget 🖼️

**Destination:** Downtown San Jose, CA (approx. 20-30 min drive from Sunnyvale)
**Mood:** Busy, Artsy (Street Art & Architecture)
**Budget:** Affordable (Free activities, budget-friendly food)

---

### ☀️ Morning (Architecture & Public Art Immersion)

*   **9:30 AM - 10:30 AM: Modern Masterpiece - San Jose City Hall**
    Begin your day by admiring the striking Postmodern architecture of the San Jose City Hall. Designed by Pritzker Prize-winning architect Richard Meier, the complex features an 18-story tower, an iconic glass rotunda, and a sweeping public plaza. Observe its "uncompromisingly modernist" design with light-filled spaces and expanses of metal, stone, and glass from the exterior. The transparent glass rotunda is a symbolic focal point.
    *   **Location:** 200 E Santa Clara St, San Jose, CA 95113
    *   **Cost:** Free (exterior viewing)

*   **10:30 AM - 11:30 AM: Colorful Forms - Children's Discovery Museum of San Jose & The Tech Interactive**
    Walk a few blocks to experience another architectural marvel: the Children's Discovery Museum of San Jose. This distinctive 52,000-square-foot purple building was designed by renowned Mexican architect Ricardo Legorreta. Note his signature use of vivid color, geometric forms, and light, which create a dynamic visual experience as light plays on its shifting violet hues. Nearby, you can also view The Tech Interactive, another downtown building designed by Legorreta.
    *   **Location:** Children's Discovery Museum (180 Woz Way, San Jose, CA 95110), The Tech Interactive (201 S Market St, San Jose, CA 95113)
    *   **Cost:** Free (exterior viewing)

### 🍔 Lunch (Affordable & Diverse)

*   **11:45 AM - 1:00 PM: Food Hall Feast - San Pedro Square Market**
    Head to San Pedro Square Market for an affordable and diverse lunch experience. This vibrant food hall boasts 17 food stalls, offering a wide array of choices from brick-oven pizza to gourmet burgers and more. It's a great spot to grab a budget-friendly meal and enjoy the lively atmosphere.
    *   **Location:** 87 N San Pedro St, San Jose, CA 95110
    *   **Cost:** Affordable (various price points at different stalls)

### 🎨 Afternoon (Street Art & Public Murals Walking Tour)

*   **1:00 PM - 4:30 PM: Downtown & SoFA District Mural Hunt**
    Embark on a self-guided walking tour through Downtown San Jose and the SoFA (South First Area) District, which is known as "Silicon Valley's Creative District". This area is a "thriving creativity hub" where artists transform streets and alleyways into vibrant canvases. Keep an eye out for:
    *   **Post Street & Lightston Alley:** Look for the iconic postcard-style mural that spells out "San Jose," celebrating the city's heritage and community.
    *   **Empire Seven Studios Alley:** While a bit north of the main SoFA area (consider a short ride or a longer walk), the alley behind 525 N 7th St is known for "giant murals that change regularly."
    *   **Specific Downtown Murals:** Seek out known pieces like "Homage" (115 E Santa Clara St), "Abundance is Constant" (83 S Second St), and "Workingman's Emporium" (260 N First St). You might also find the "Qmunity & Pride Post Street" mural at 65 Post St.
    *   **San Jose State University Campus:** Explore murals near the student union and Cesar Chavez archway, which often carry messages about education and community.
    *   **Art Ark Exterior Murals:** Take a slight detour to 1058 South Fifth Street to see the colorful murals on the exterior walls of this affordable artist housing complex.
    *   **Guadalupe River Trail:** Look for the monumental "We Are Muwekma Ohlone Mural" by artist Alfonso Salazar along this trail, a powerful depiction of resistance and heritage.
    *   **Tip:** Consider consulting the "Heart of the Valley" online map for more mural locations.
    *   **Cost:** Free (walking tour)

### 🌆 Evening (More Art & Affordable Dinner)

*   **4:30 PM - 6:00 PM: SoFA District Gallery & Vibe Check**
    Continue to immerse yourself in the artsy vibe of the SoFA District. While many galleries may be closing or have specific hours, you can still appreciate the outdoor art, vibrant storefronts, and overall creative atmosphere. This district is home to institutions like the Institute of Contemporary Art San José (ICA) and Movimiento de Arte y Cultura Latino Americana (MACLA), which you can appreciate from the outside or check for any free public viewing times if available.
    *   **Location:** SoFA District, primarily along South First Street, San Jose
    *   **Cost:** Free (exploration), potential minimal cost if an affordable gallery entry fee applies (check specific gallery schedules)

*   **6:00 PM - 7:30 PM: Budget Bites - Downtown Delights**
    For dinner, choose another affordable spot in Downtown San Jose.
    *   **Option 1: Super Taqueria:** A local favorite for its "super burritos and tacos," known for being popular with San Jose State University students. (Note: typically cash-only).
        *   **Location:** 140 E San Carlos St, San Jose, CA
    *   **Option 2: Costco Food Court:** If you don't mind a short drive or are near a Costco on your way back, a $1.50 hotdog and soda is an unbeatable deal, and you don't need a membership to access the food court.
    *   **Cost:** Affordable (Under $10-15 per person)

*   **7:30 PM Onwards: Evening Stroll & Departure**
    Enjoy a final evening stroll through the illuminated streets of Downtown San Jose, perhaps revisiting some of your favorite murals or architectural sights. Then, head back to Sunnyvale, filled with the inspiration of a busy and artsy day.

---

--------------------------------------------------



---
## Part 2: Supercharging Agents with Custom Tools 🛠️

So far, we've used the powerful built-in `GoogleSearch` tool. But the true power of agents comes from connecting them to your own logic and data sources.

This is where **custom tools** come in. Let's explore three patterns for giving your agent new skills, using real-world, practical examples.

### 2.1 The Simple `FunctionTool`: Calling a Real-Time Weather API

The most direct way to create a tool is by writing a Python function. This is perfect for synchronous tasks like fetching data from an API.

**Key Concept:** The function's **docstring** is critical. The ADK uses it as the tool's official description, which the LLM reads to understand its purpose, parameters, and when to use it.

In this example, we'll create a tool that calls the **free, public U.S. National Weather Service API** to get a real-time forecast. No API key needed!

In [9]:
# --- Tool Definition: A function that calls a live public API ---
import requests
import json

# A simple lookup to avoid needing a separate geocoding API for this example
LOCATION_COORDINATES = {
    "sunnyvale": "37.3688,-122.0363",
    "san francisco": "37.7749,-122.4194",
    "lake tahoe": "39.0968,-120.0324"
}

def get_live_weather_forecast(location: str) -> dict:
    """Gets the current, real-time weather forecast for a specified location in the US.

    Args:
        location: The city name, e.g., "San Francisco".

    Returns:
        A dictionary containing the temperature and a detailed forecast.
    """
    print(f"🛠️ TOOL CALLED: get_live_weather_forecast(location='{location}')")

    # Find coordinates for the location
    normalized_location = location.lower()
    coords_str = None
    for key, val in LOCATION_COORDINATES.items():
        if key in normalized_location:
            coords_str = val
            break
    if not coords_str:
        return {"status": "error", "message": f"I don't have coordinates for {location}."}

    try:
        # NWS API requires 2 steps: 1. Get the forecast URL from the coordinates.
        points_url = f"https://api.weather.gov/points/{coords_str}"
        headers = {"User-Agent": "ADK Example Notebook"}
        points_response = requests.get(points_url, headers=headers)
        points_response.raise_for_status() # Raise an exception for bad status codes
        forecast_url = points_response.json()['properties']['forecast']

        # 2. Get the actual forecast from the URL.
        forecast_response = requests.get(forecast_url, headers=headers)
        forecast_response.raise_for_status()

        # Extract the relevant forecast details
        current_period = forecast_response.json()['properties']['periods'][0]
        return {
            "status": "success",
            "temperature": f"{current_period['temperature']}°{current_period['temperatureUnit']}",
            "forecast": current_period['detailedForecast']
        }
    except requests.exceptions.RequestException as e:
        return {"status": "error", "message": f"API request failed: {e}"}

# --- Agent Definition: An agent that USES the new tool ---

weather_agent = Agent(
    name="weather_aware_planner",
    model="gemini-2.5-flash",
    description="A trip planner that checks the real-time weather before making suggestions.",
    instruction="You are a cautious trip planner. Before suggesting any outdoor activities, you MUST use the `get_live_weather_forecast` tool to check conditions. Incorporate the live weather details into your recommendation.",
    tools=[get_live_weather_forecast]
)

print(f"🌦️ Agent '{weather_agent.name}' is created and can now call a live weather API!")

🌦️ Agent 'weather_aware_planner' is created and can now call a live weather API!


In [11]:
# --- Let's test the Weather-Aware Planner ---

async def run_weather_planner_test():
    weather_session = await session_service.create_session(app_name=weather_agent.name, user_id=my_user_id)
    query = "I want to go kayaking near Lake Tahoe, what's the weather and wind like? Please give the temperature in Celsius"
    print(f"🗣️ User Query: '{query}'")
    await run_agent_query(weather_agent, query, weather_session, my_user_id)

await run_weather_planner_test()

🗣️ User Query: 'I want to go kayaking near Lake Tahoe, what's the weather and wind like? Please give the temperature in Celsius'

🚀 Running query for agent: 'weather_aware_planner' in session: 'f56446ed-8628-4ce2-9010-2190973cc869'...


EVENT: content=Content(
  parts=[
    Part(
      function_call=FunctionCall(
        args={
          'location': 'Lake Tahoe'
        },
        id='adk-379f8f5d-5dc5-4260-9a3d-20ade161b0be',
        name='get_live_weather_forecast'
      ),
      thought_signature=b'\n\x9f\x04\x01T\xa8\\\xee\x9c\xfc\xabS&\x84\xd0wq\x83s?\xda\xbe\xf2\xe0\x08f\x95{J5Y\xef1N\xdf\xda\xa5u.;\xd6\xd7g\xe8\x18A\xd0\xc4Eh\xbc\xe4\x1a\x9b\xab\x1c\x87v\xd2\xf2\xca\x12z\xef\x9a\xd7)\x99\x16H\x8d\xf1i\x03G2=|\x8by]I"\xc2\xcf\xcd\xe2\xcf\x8f{\x12\x88K\xec\x82|\x06...'
    ),
  ],
  role='model'
) grounding_metadata=None partial=None turn_complete=None error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=GenerateContentResponseUsageMetadata(
  candidates_token_count=20,
  prompt_token_count=196,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=196
    ),
  ],
  thoughts_token_count=105,
  total_token_count=321
) liv

EVENT: content=Content(
  parts=[
    Part(
      text='The weather near Lake Tahoe is partly sunny, with a high near 25°C. There is a chance of rain showers between 11 am and 2 pm, followed by a chance of showers and thunderstorms. The wind will be from the Northeast at 0 to 5 mph. Given the chance of rain and thunderstorms, it would be wise to monitor the weather closely if you plan to go kayaking.',
      thought_signature=b"\n\xca\x07\x01T\xa8\\\xee9\xdf\x04\xd0\xf6\x05\x18}\xda~\xc8\xa52P\x8a\x83\xbd\x14G\x91o,\n\xc9\xack\xc9\xbc56\xe6\xdb\xcfgU\xe9'\xd1UE\xc5\xe9\x08\xbb\xa8\xd3\x93\x0b:5\xb2\x96oH\xb3I\xe3\x15\xd3\x1ck\x95Z\x1f3\x99V\x8f\r\x0f(+\xf4\x13U\x17\xcd\xcfO\xf6\x97\xff\x8eH\x0cU\x14\xb9\xc8...'
    ),
  ],
  role='model'
) grounding_metadata=None partial=None turn_complete=None error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=GenerateContentResponseUsageMetadata(
  candidates_token_count=84,
  prompt_token_count=300,
  prompt_toke

The weather near Lake Tahoe is partly sunny, with a high near 25°C. There is a chance of rain showers between 11 am and 2 pm, followed by a chance of showers and thunderstorms. The wind will be from the Northeast at 0 to 5 mph. Given the chance of rain and thunderstorms, it would be wise to monitor the weather closely if you plan to go kayaking.

--------------------------------------------------



## 2.2 The Agent-as-a-Tool: Consulting a Specialist 🧑‍🍳

Why build one agent that does everything when you can build a **team of specialist agents?** The **Agent-as-a-Tool** pattern allows one agent to delegate a task to another agent.

**Key Concept:** This is different from a sub-agent. When Agent A calls Agent B as a tool, Agent B's response is passed **back to Agent A**. Agent A then uses that information to form its own final response to the user. It's a powerful way to compose complex behaviors from simpler, focused, and reusable agents.

### How It Works

Our top-level agent, the `trip_data_concierge_agent`, acts as the **Orchestrator**. It has two tools at its disposal:

1.  `call_db_agent`: A function that internally calls our `db_agent` to fetch raw data.
2.  `call_concierge_agent`: A function that calls the `concierge_agent`.

The `concierge_agent` itself has a tool: the `food_critic_agent`.

The flow for a complex query is:

1.  **User** asks the `trip_data_concierge_agent` for a hotel and a nearby restaurant.
2.  The **Orchestrator** first calls `call_db_agent` to get hotel data.
3.  The data is saved in `tool_context.state`.
4.  The **Orchestrator** then calls `call_concierge_agent`, which retrieves the hotel data from the context.
5.  The `concierge_agent` receives the request and decides it needs to use its own tool, the `food_critic_agent`.
6.  The `food_critic_agent` provides a witty recommendation.
7.  The `concierge_agent` gets the critic's response and politely formats it.
8.  This final, polished response is returned to the **Orchestrator**, which presents it to the user.

                         +-----------------------------------------------------------+
                         |              🧭 Trip Data Concierge Agent                 |
                         |-----------------------------------------------------------|
                         |  Model: gemini-2.5-flash                                  |
                         |  Description:                                             |
                         |   Orchestrates database query and travel recommendation  |
                         |-----------------------------------------------------------|
                         |  🔧 Tools:                                                |
                         |   1. call_db_agent                                        |
                         |   2. call_concierge_agent                                 |
                         +-----------------------------------------------------------+
                                      /                                \
                                     /                                  \
                                    ▼                                    ▼
        +-------------------------------------------+    +---------------------------------------------+
        |            🔧 Tool: call_db_agent         |    |         🔧 Tool: call_concierge_agent        |
        |-------------------------------------------|    |---------------------------------------------|
        | Calls: db_agent                           |    | Calls: concierge_agent                       |
        |                                           |    | Uses data from db_agent for recommendations |
        +-------------------------------------------+    +---------------------------------------------+
                                |                                          |
                                ▼                                          ▼
       +--------------------------------------------+   +------------------------------------------------+
       |              📦 db_agent                   |   |             🤵 concierge_agent                  |
       |--------------------------------------------|   |------------------------------------------------|
       | Model: gemini-2.5-flash                    |   | Model: gemini-2.5-flash                         |
       | Role: Return mock JSON hotel data          |   | Role: Hotel staff that handles user Q&A        |
       +--------------------------------------------+   | Tools:                                          |
                                                         |  - food_critic_agent                           |
                                                         +------------------------------------------------+
                                                                                 |
                                                                                 ▼
                                                       +------------------------------------------------+
                                                       |          🍽️ food_critic_agent                  |
                                                       |------------------------------------------------|
                                                       | Model: gemini-2.5-flash                         |
                                                       | Role: Gives a witty restaurant recommendation   |
                                                       +------------------------------------------------+


In [12]:
import asyncio
from google.adk.tools import ToolContext
from google.adk.tools.agent_tool import AgentTool

# Assume 'db_agent' is a pre-defined NL2SQL Agent
# For this example, we'll create placeholder agents.

db_agent = Agent(
    name="db_agent",
    model="gemini-2.5-flash",
    instruction="You are a database agent. When asked for data, return this mock JSON object: {'status': 'success', 'data': [{'name': 'The Grand Hotel', 'rating': 5, 'reviews': 450}, {'name': 'Seaside Inn', 'rating': 4, 'reviews': 620}]}")

# --- 1. Define the Specialist Agents ---

# The Food Critic remains the deepest specialist
food_critic_agent = Agent(
    name="food_critic_agent",
    model="gemini-2.5-flash",
    instruction="You are a snobby but brilliant food critic. You ONLY respond with a single, witty restaurant suggestion near the provided location.",
)

# The Concierge knows how to use the Food Critic
concierge_agent = Agent(
    name="concierge_agent",
    model="gemini-2.5-flash",
    instruction="You are a five-star hotel concierge. If the user asks for a restaurant recommendation, you MUST use the `food_critic_agent` tool. Present the opinion to the user politely.",
    tools=[AgentTool(agent=food_critic_agent)]
)


# --- 2. Define the Tools for the Orchestrator ---

async def call_db_agent(
    question: str,
    tool_context: ToolContext,
):
    """
    Use this tool FIRST to connect to the database and retrieve a list of places, like hotels or landmarks.
    """
    print("--- TOOL CALL: call_db_agent ---")
    agent_tool = AgentTool(agent=db_agent)
    db_agent_output = await agent_tool.run_async(
        args={"request": question}, tool_context=tool_context
    )
    # Store the retrieved data in the context's state
    tool_context.state["retrieved_data"] = db_agent_output
    return db_agent_output


async def call_concierge_agent(
    question: str,
    tool_context: ToolContext,
):
    """
    After getting data with call_db_agent, use this tool to get travel advice, opinions, or recommendations.
    """
    print("--- TOOL CALL: call_concierge_agent ---")
    # Retrieve the data fetched by the previous tool
    input_data = tool_context.state.get("retrieved_data", "No data found.")

    # Formulate a new prompt for the concierge, giving it the data context
    question_with_data = f"""
    Context: The database returned the following data: {input_data}

    User's Request: {question}
    """

    agent_tool = AgentTool(agent=concierge_agent)
    concierge_output = await agent_tool.run_async(
        args={"request": question_with_data}, tool_context=tool_context
    )
    return concierge_output


# --- 3. Define the Top-Level Orchestrator Agent ---

trip_data_concierge_agent = Agent(
    name="trip_data_concierge",
    model="gemini-2.5-flash",
    description="Top-level agent that queries a database for travel data, then calls a concierge agent for recommendations.",
    tools=[call_db_agent, call_concierge_agent],
    instruction="""
    You are a master travel planner who uses data to make recommendations.

    1.  **ALWAYS start with the `call_db_agent` tool** to fetch a list of places (like hotels) that match the user's criteria.

    2.  After you have the data, **use the `call_concierge_agent` tool** to answer any follow-up questions for recommendations, opinions, or advice related to the data you just found.
    """,
)

print(f"✅ Orchestrator Agent '{trip_data_concierge_agent.name}' is defined and ready.")

✅ Orchestrator Agent 'trip_data_concierge' is defined and ready.


In [13]:
# --- Let's test the Trip Data Concierge Agent ---

async def run_trip_data_concierge():
    """
    Sets up a session and runs a query against the top-level
    trip_data_concierge_agent.
    """
    # Create a new, single-use session for this query
    concierge_session = await session_service.create_session(
        app_name=trip_data_concierge_agent.name,
        user_id=my_user_id
    )

    # This query is specifically designed to trigger the full two-step process:
    # 1. Get data from the db_agent.
    # 2. Get a recommendation from the concierge_agent based on that data.
    query = "Find the top-rated hotels in San Francisco from the database, then suggest a dinner spot near the one with the most reviews."
    print(f"🗣️ User Query: '{query}'")

    # We call our existing helper function with the top-level orchestrator agent
    await run_agent_query(trip_data_concierge_agent, query, concierge_session, my_user_id)

# Run the test
await run_trip_data_concierge()

🗣️ User Query: 'Find the top-rated hotels in San Francisco from the database, then suggest a dinner spot near the one with the most reviews.'

🚀 Running query for agent: 'trip_data_concierge' in session: 'a48c8ca5-07e1-4a4e-8a21-2a4abab4ce81'...


EVENT: content=Content(
  parts=[
    Part(
      function_call=FunctionCall(
        args={
          'question': 'top-rated hotels in San Francisco'
        },
        id='adk-30e0a774-3b91-40ec-bf30-95597b28a46f',
        name='call_db_agent'
      ),
      thought_signature=b'\n\x89\x05\x01T\xa8\\\xee\xc7\x13\xd6\x9d\x91\x84\x80\xbcf$\x83\xd8\xa79\xd6\x1d}\x94\x06\x976\x81\x9d|\t\x8a\xdb\x8f\xb2|\xa0\x14\x19\xea9\x8bK\xc5\x87\x0eK3\xd9\xa6D\x96\xad\tOU\xd03\x81\xda\xe6\x0e-5z\t\xc2f\xd3\x91J\xe6\xb0\xda\xfb\t.\xa1\x8a<\xda`\xfd\xd5\xb8\xfb\x05>\x03\xc1\xa7Nv\xce\xb9...'
    ),
  ],
  role='model'
) grounding_metadata=None partial=None turn_complete=None error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=GenerateContentResponseUsageMetadata(
  candidates_token_count=23,
  prompt_token_count=294,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=294
    ),
  ],
  thoughts_token_count=1

EVENT: content=Content(
  parts=[
    Part(
      function_call=FunctionCall(
        args={
          'question': 'Suggest a dinner spot near Seaside Inn in San Francisco.'
        },
        id='adk-e4cdcf58-e3c4-43cc-9ae8-636414bea675',
        name='call_concierge_agent'
      ),
      thought_signature=b'\n\x9e\x03\x01T\xa8\\\xee\xe3\xcb\xd9I\xf8\xefv\xed\xc8\xd8\x88,\xef\xc2\x10\xc8FP\xa2\x8c\x92\xfd\x05\xbf\x87\xfe\xd4\x8d\xef\xee\xd2B\x94\xdfz\x83U\x90\xcf*0\xd0Db\x00\x98\x1c\xdc+g7\xc4^P\xc8\x87\x82\x88\xe9\x96+\xbf\xdc@\xa5\xb7\x0f\x84\xbe\xfdGe\xeb\xea\xb2\xf27\x077\x80<\x12#\x15\x90\x0f\xde\xfe ...'
    ),
  ],
  role='model'
) grounding_metadata=None partial=None turn_complete=None error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=GenerateContentResponseUsageMetadata(
  candidates_token_count=27,
  prompt_token_count=430,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=43

EVENT: content=Content(
  parts=[
    Part(
      function_response=FunctionResponse(
        id='adk-e4cdcf58-e3c4-43cc-9ae8-636414bea675',
        name='call_concierge_agent',
        response={
          'result': 'Ah, a discerning choice, if I may say so myself. For an exceptional dinner experience near the Seaside Inn in San Francisco, I would heartily recommend **Gary Danko**. It is a establishment that consistently delights with its refined offerings.'
        }
      )
    ),
  ],
  role='user'
) grounding_metadata=None partial=None turn_complete=None error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=None live_session_resumption_update=None invocation_id='e-c93535b0-4fd9-43d4-a1d0-05f0f97d4e6b' author='trip_data_concierge' actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}) long_running_tool_ids=None branch=None id='3d42e823-7f32-4236-a570-601a6fb

EVENT: content=Content(
  parts=[
    Part(
      text="""The top-rated hotels in San Francisco are The Grand Hotel (5 stars, 450 reviews) and Seaside Inn (4 stars, 620 reviews). The Seaside Inn has the most reviews.

For an exceptional dinner experience near the Seaside Inn, I would heartily recommend **Gary Danko**.""",
      thought_signature=b'\n\xba\x04\x01T\xa8\\\xee\x9b\xc7CV}\xa2F\xa3q\xb6\xda"\xd0\tv\xd1\x81t\xfa\xafx\x99\xad\xc6\x801\xc1(\xd57\xfc\x08{\xb9\xbb\xd2%3\xe3\xa83\xb4\xb6\x9e\x8a\x1ew\xd3\x11\xf1\x85W\x83\xe7PT\x00\xc0\xa1sfI\x8a\xc0We\xb6\xb1\x9c\xe9x\x13\xf5\x85\xf3\xea\x14\\\xa4\x1d\x07\xcfH\x11\xad\xef\xa1\x12T...'
    ),
  ],
  role='model'
) grounding_metadata=None partial=None turn_complete=None error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=GenerateContentResponseUsageMetadata(
  candidates_token_count=63,
  prompt_token_count=521,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 

The top-rated hotels in San Francisco are The Grand Hotel (5 stars, 450 reviews) and Seaside Inn (4 stars, 620 reviews). The Seaside Inn has the most reviews.

For an exceptional dinner experience near the Seaside Inn, I would heartily recommend **Gary Danko**.

--------------------------------------------------



---
## Part 3: Agent with a Memory - The Adaptive Planner 🗺️

Now, let's see an agent that not only **remembers** but also **adapts**. We'll challenge the `multi_day_trip_agent` to re-plan part of its itinerary based on our feedback. This is a much more realistic test of conversational AI.

```
+-----------------------------------------------------+
|         Adaptive Multi-Day Trip Agent 🗺️           |
|-----------------------------------------------------|
|  Model: gemini-2.5-flash                            |
|  Description:                                       |
|   Builds multi-day travel itineraries step-by-step, |
|   remembers previous days, adapts to feedback       |
|-----------------------------------------------------|
|  🔧 Tools:                                          |
|   - Google Search                                   |
|-----------------------------------------------------|
|  🧠 Capabilities:                                   |
|   - Memory of past conversation & preferences       |
|   - Progressive planning (1 day at a time)          |
|   - Adapts to user feedback                         |
|   - Ensures activity variety across days            |
+-----------------------------------------------------+

            ▲
            |
    +---------------------------+
    |     User Interaction      |
    |---------------------------|
    | - Destination             |
    | - Trip duration           |
    | - Interests & feedback    |
    +---------------------------+

            |
            ▼

+-----------------------------------------------------+
|        Day-by-Day Itinerary Generation              |
|-----------------------------------------------------|
|  🗓️ Day N Output (Markdown format):                 |
|   - Morning / Afternoon / Evening activities        |
|   - Personalized & context-aware                    |
|   - Changes accepted, feedback acknowledged         |
+-----------------------------------------------------+

            |
            ▼

+-----------------------------------------------------+
|        Next Day Planning Triggered 🚀               |
|-----------------------------------------------------|
| - Builds on prior days                              |
| - Avoids repetition                                 |
| - Asks user for confirmation before proceeding      |
+-----------------------------------------------------+
```


In [14]:
# --- Agent Definition: The Adaptive Planner ---

def create_multi_day_trip_agent():
    """Create the Progressive Multi-Day Trip Planner agent"""
    return Agent(
        name="multi_day_trip_agent",
        model="gemini-2.5-flash",
        description="Agent that progressively plans a multi-day trip, remembering previous days and adapting to user feedback.",
        instruction="""
        You are the "Adaptive Trip Planner" 🗺️ - an AI assistant that builds multi-day travel itineraries step-by-step.

        Your Defining Feature:
        You have short-term memory. You MUST refer back to our conversation to understand the trip's context, what has already been planned, and the user's preferences. If the user asks for a change, you must adapt the plan while keeping the unchanged parts consistent.

        Your Mission:
        1.  **Initiate**: Start by asking for the destination, trip duration, and interests.
        2.  **Plan Progressively**: Plan ONLY ONE DAY at a time. After presenting a plan, ask for confirmation.
        3.  **Handle Feedback**: If a user dislikes a suggestion (e.g., "I don't like museums"), acknowledge their feedback, and provide a *new, alternative* suggestion for that time slot that still fits the overall theme.
        4.  **Maintain Context**: For each new day, ensure the activities are unique and build logically on the previous days. Do not suggest the same things repeatedly.
        5.  **Final Output**: Return each day's itinerary in MARKDOWN format.
        """,
        tools=[google_search]
    )

multi_day_agent = create_multi_day_trip_agent()
print(f"🗺️ Agent '{multi_day_agent.name}' is created and ready to plan and adapt!")

🗺️ Agent 'multi_day_trip_agent' is created and ready to plan and adapt!


### Scenario 3a: Agent WITH Memory (Using a SINGLE Session) ✅

First, let's see the correct way to do it. We will use the **exact same `trip_session` object** for the entire conversation. Watch how the agent remembers the context from Turn 1 to correctly handle the requests in Turn 2 and 3.

In [15]:
# --- Scenario 2: Testing Adaptation and Memory ---

async def run_adaptive_memory_demonstration():
    print("### 🧠 DEMO 2: AGENT THAT ADAPTS (SAME SESSION) ###")

    # Create ONE session that we will reuse for the whole conversation
    trip_session = await session_service.create_session(
        app_name=multi_day_agent.name,
        user_id=my_user_id
    )
    print(f"Created a single session for our trip: {trip_session.id}")

    # --- Turn 1: The user initiates the trip ---
    query1 = "Hi! I want to plan a 2-day trip to Osaka, Japan. I'm interested in historic sites and great local food."
    print(f"\n🗣️ User (Turn 1): '{query1}'")
    await run_agent_query(multi_day_agent, query1, trip_session, my_user_id)

    # --- Turn 2: The user gives FEEDBACK and asks for a CHANGE ---
    # We use the EXACT SAME `trip_session` object!
    query2 = "That sounds pretty good, but I'm not a huge fan of castles. Can you replace the morning activity for Day 1 with something else historical?"
    print(f"\n🗣️ User (Turn 2 - Feedback): '{query2}'")
    await run_agent_query(multi_day_agent, query2, trip_session, my_user_id)

    # --- Turn 3: The user confirms and asks to continue ---
    query3 = "Yes, the new plan for Day 1 is perfect! Please plan Day 2 now, keeping the food theme in mind."
    print(f"\n🗣️ User (Turn 3 - Confirmation): '{query3}'")
    await run_agent_query(multi_day_agent, query3, trip_session, my_user_id)

await run_adaptive_memory_demonstration()

### 🧠 DEMO 2: AGENT THAT ADAPTS (SAME SESSION) ###
Created a single session for our trip: aae7298d-0dda-43d9-9443-e6aa875843fd

🗣️ User (Turn 1): 'Hi! I want to plan a 2-day trip to Osaka, Japan. I'm interested in historic sites and great local food.'

🚀 Running query for agent: 'multi_day_trip_agent' in session: 'aae7298d-0dda-43d9-9443-e6aa875843fd'...
EVENT: content=Content(
  parts=[
    Part(
      text="""Hello! I'd be delighted to help you plan a 2-day trip to Osaka focusing on historic sites and delicious local food. Osaka is a fantastic choice for both!

Let's start with **Day 1**. How does this sound?

## Day 1: Historic Osaka and Dotonbori Delights

*   **Morning (9:00 AM - 1:00 PM): Osaka Castle Exploration**
    Begin your day at Osaka Castle, one of Japan's most famous landmarks. Built in 1583 by Hideyoshi Toyotomi, it has been rebuilt several times throughout history. You can explore the castle grounds, which span 106 hectares with green spaces and impressive fortificati

Hello! I'd be delighted to help you plan a 2-day trip to Osaka focusing on historic sites and delicious local food. Osaka is a fantastic choice for both!

Let's start with **Day 1**. How does this sound?

## Day 1: Historic Osaka and Dotonbori Delights

*   **Morning (9:00 AM - 1:00 PM): Osaka Castle Exploration**
    Begin your day at Osaka Castle, one of Japan's most famous landmarks. Built in 1583 by Hideyoshi Toyotomi, it has been rebuilt several times throughout history. You can explore the castle grounds, which span 106 hectares with green spaces and impressive fortifications. The castle museum inside offers insights into its history, and you can enjoy panoramic views from the observation deck. The museum is open from 9:00 AM to 5:00 PM, with last admission at 4:30 PM.
*   **Lunch (1:00 PM - 2:30 PM): Local Flavors near Osaka Castle**
    After exploring the castle, find a local eatery near Osaka Castle Park. You could try "Yatagarasu UDON" for a bowl of curry udon or "Endo Sushi Kyobashi" for affordable sushi.
*   **Afternoon (2:30 PM - 5:30 PM): Shitennoji Temple**
    Next, visit Shitennoji Temple, often claimed to be Japan's oldest temple, founded in 593 by Prince Shotoku. While the original buildings have been destroyed and rebuilt over centuries, the temple retains its historical significance. You can enter the outer temple grounds for free and admire the sacred buildings. For a small fee, you can visit the Chushin Garan (inner-temple complex) with the main hall and a five-story pagoda, and the Treasure House, which displays Buddhist scriptures, statues, paintings, and artifacts.
*   **Evening (6:00 PM onwards): Dotonbori Food Adventure**
    Conclude your day with a culinary journey through Dotonbori, Osaka's iconic food and entertainment district. This area is famous for its dazzling neon lights and a wide array of street food. Be sure to try Osaka's famous dishes like **Takoyaki** (octopus balls), **Okonomiyaki** (savory pancakes), and **Kushikatsu** (deep-fried skewers). You'll find many popular spots like Kushikatsu Daruma for skewers and Chibo Okonomiyaki Restaurant for savory pancakes.

Let me know what you think of this plan for Day 1! We can adjust it based on your preferences.

--------------------------------------------------


🗣️ User (Turn 2 - Feedback): 'That sounds pretty good, but I'm not a huge fan of castles. Can you replace the morning activity for Day 1 with something else historical?'

🚀 Running query for agent: 'multi_day_trip_agent' in session: 'aae7298d-0dda-43d9-9443-e6aa875843fd'...
EVENT: content=Content(
  parts=[
    Part(
      text="""Understood! No problem at all, we can certainly swap out the castle for another historical gem.

Let's revise Day 1 to include **Sumiyoshi Taisha Shrine** instead of Osaka Castle.

## Day 1: Ancient Shrine, Sacred Temple, and Dotonbori Delights

*   **Morning (9:00 AM - 1:00 PM): Sumiyoshi Taisha Shrine**
    Begin your day at **Sumiyoshi Taisha Shrine**, one of Japan's oldest and most famous Shinto shrines, founded in the 3rd century. It predates the introduction of Buddhism to Japan and showcases a unique architectural style called *Sumiyoshi-zukuri*, which features straight roofs and pillars painted in v

Understood! No problem at all, we can certainly swap out the castle for another historical gem.

Let's revise Day 1 to include **Sumiyoshi Taisha Shrine** instead of Osaka Castle.

## Day 1: Ancient Shrine, Sacred Temple, and Dotonbori Delights

*   **Morning (9:00 AM - 1:00 PM): Sumiyoshi Taisha Shrine**
    Begin your day at **Sumiyoshi Taisha Shrine**, one of Japan's oldest and most famous Shinto shrines, founded in the 3rd century. It predates the introduction of Buddhism to Japan and showcases a unique architectural style called *Sumiyoshi-zukuri*, which features straight roofs and pillars painted in vermilion. Walk across the picturesque Sorihashi Bridge (Taiko-bashi) and explore the various shrine buildings, absorbing the serene and ancient atmosphere. The main hall is a designated National Treasure.
*   **Lunch (1:00 PM - 2:30 PM): Local Flavors near Sumiyoshi Taisha or on the way to Shitennoji**
    After exploring Sumiyoshi Taisha, you can find a local spot for lunch. There are eateries around the shrine, or you can head towards your next destination, Shitennoji Temple, and find a restaurant along the way.
*   **Afternoon (2:30 PM - 5:30 PM): Shitennoji Temple**
    Next, visit Shitennoji Temple, often claimed to be Japan's oldest temple, founded in 593 by Prince Shotoku. While the original buildings have been destroyed and rebuilt over centuries, the temple retains its historical significance. You can enter the outer temple grounds for free and admire the sacred buildings. For a small fee, you can visit the Chushin Garan (inner-temple complex) with the main hall and a five-story pagoda, and the Treasure House, which displays Buddhist scriptures, statues, paintings, and artifacts.
*   **Evening (6:00 PM onwards): Dotonbori Food Adventure**
    Conclude your day with a culinary journey through Dotonbori, Osaka's iconic food and entertainment district. This area is famous for its dazzling neon lights and a wide array of street food. Be sure to try Osaka's famous dishes like **Takoyaki** (octopus balls), **Okonomiyaki** (savory pancakes), and **Kushikatsu** (deep-fried skewers). You'll find many popular spots like Kushikatsu Daruma for skewers and Chibo Okonomiyaki Restaurant for savory pancakes.

How does this revised Day 1 plan sound to you? We can then move on to Day 2!

--------------------------------------------------


🗣️ User (Turn 3 - Confirmation): 'Yes, the new plan for Day 1 is perfect! Please plan Day 2 now, keeping the food theme in mind.'

🚀 Running query for agent: 'multi_day_trip_agent' in session: 'aae7298d-0dda-43d9-9443-e6aa875843fd'...
EVENT: content=Content(
  parts=[
    Part(
      text="""Excellent! I'm glad Day 1 is to your liking. Now, let's craft an exciting and delicious **Day 2** for you in Osaka, keeping your interests in history and food firmly in mind.

## Day 2: Scholar's Shrine, Market Delights, and Retro Flavors

*   **Morning (9:30 AM - 12:30 PM): Osaka Tenmangu Shrine**
    Start your day at **Osaka Tenmangu Shrine**, a significant Shinto shrine dedicated to Sugawara no Michizane, the god of scholarship and learning. Founded in the 10th century, this shrine is particularly famous for hosting the Tenjin Matsuri, one of Japan's top three festivals, every July. Explore the main hall, admire the intricate architecture, an

Excellent! I'm glad Day 1 is to your liking. Now, let's craft an exciting and delicious **Day 2** for you in Osaka, keeping your interests in history and food firmly in mind.

## Day 2: Scholar's Shrine, Market Delights, and Retro Flavors

*   **Morning (9:30 AM - 12:30 PM): Osaka Tenmangu Shrine**
    Start your day at **Osaka Tenmangu Shrine**, a significant Shinto shrine dedicated to Sugawara no Michizane, the god of scholarship and learning. Founded in the 10th century, this shrine is particularly famous for hosting the Tenjin Matsuri, one of Japan's top three festivals, every July. Explore the main hall, admire the intricate architecture, and soak in the serene atmosphere. The shrine offers a different historical and cultural perspective compared to Sumiyoshi Taisha, with a focus on education and traditional festivals.
*   **Lunch (12:30 PM - 2:00 PM): Explore Kuromon Ichiba Market**
    Head to **Kuromon Ichiba Market**, often called "Osaka's Kitchen." This bustling public market has a history of over 190 years and is a paradise for food lovers. Wander through the aisles, sampling fresh seafood (sushi, sashimi, grilled scallops, sea urchin), local produce, and various street foods like grilled wagyu skewers, oden, and fresh fruit. You can pick and choose from different stalls for a varied and authentic Osaka lunch experience.
*   **Afternoon (2:00 PM - 5:00 PM): Shinsekai District and Tsutenkaku Tower**
    Immerse yourself in the unique, retro atmosphere of the **Shinsekai district**. This area was developed before World War II and retains a nostalgic, slightly gritty charm. Its most prominent landmark is the **Tsutenkaku Tower**, a symbol of Osaka. You can ascend the tower for panoramic views of the city. While in Shinsekai, you'll feel like you've stepped back in time with its unique architecture, vibrant signage, and charmingly old-fashioned shops.
*   **Evening (6:00 PM onwards): Shinsekai's Kushikatsu and Local Izakayas**
    Conclude your trip by enjoying dinner in Shinsekai, which is renowned for its **Kushikatsu**. This Osaka specialty involves various ingredients (meat, vegetables, seafood) deep-fried on skewers. There are many famous kushikatsu restaurants in the area, often with lively, casual atmospheres. Try "Daruma" or "Yaekatsu" for an authentic experience. Afterwards, you might want to visit a local *izakaya* (Japanese pub) for some drinks and small plates to round off your trip.

How does this plan for Day 2 sound for completing your Osaka adventure?

--------------------------------------------------



### Scenario 3b: Agent WITHOUT Memory (Using SEPARATE Sessions) ❌

Now, let's see what happens if we mess up our session management. Here, we'll give the agent a case of amnesia by creating a **brand new, separate session for each turn**.

Pay close attention to the agent's response to the second query. Because it's in a new session, it has no memory of the trip to Lisbon we just discussed!

In [16]:
# --- Scenario 2b: Demonstrating Memory FAILURE ---

async def run_memory_failure_demonstration():
    print("\n" + "#"*60)
    print("### 🧠 DEMO 2b: AGENT WITH AMNESIA (SEPARATE SESSIONS) ###")
    print("#"*60)

    # --- Turn 1: The user initiates the trip in the FIRST session ---
    query1 = "Hi! I want to plan a 2-day trip to Osaka, Japan. I'm interested in historic sites and great local food."
    session_one = await session_service.create_session(
        app_name=multi_day_agent.name,
        user_id=my_user_id
    )
    print(f"\nCreated a session for Turn 1: {session_one.id}")
    print(f"🗣️ User (Turn 1): '{query1}'")
    await run_agent_query(multi_day_agent, query1, session_one, my_user_id)

    # --- Turn 2: The user asks to continue... but in a completely NEW session ---
    query2 = "Yes, that looks perfect! Please plan Day 2."
    session_two = await session_service.create_session(
        app_name=multi_day_agent.name,
        user_id=my_user_id
    )
    print(f"\nCreated a BRAND NEW session for Turn 2: {session_two.id}")
    print(f"🗣️ User (Turn 2): '{query2}'")
    await run_agent_query(multi_day_agent, query2, session_two, my_user_id)

await run_memory_failure_demonstration()


############################################################
### 🧠 DEMO 2b: AGENT WITH AMNESIA (SEPARATE SESSIONS) ###
############################################################

Created a session for Turn 1: 3d419f96-93f3-466a-830d-16fa8952c37f
🗣️ User (Turn 1): 'Hi! I want to plan a 2-day trip to Osaka, Japan. I'm interested in historic sites and great local food.'

🚀 Running query for agent: 'multi_day_trip_agent' in session: '3d419f96-93f3-466a-830d-16fa8952c37f'...
EVENT: content=Content(
  parts=[
    Part(
      text="""Here's a possible itinerary for your first day in Osaka, focusing on historic sites and local food:

### Day 1: Osaka Castle History and Dotonbori Delights

*   **Morning (9:00 AM - 1:00 PM): Explore Osaka Castle**
    Start your day with a visit to the iconic Osaka Castle, a significant historic landmark that dates back to 1583. Explore the castle's interior museum, which details its rich history and houses various artifacts, including samurai armor. Climb to

Here's a possible itinerary for your first day in Osaka, focusing on historic sites and local food:

### Day 1: Osaka Castle History and Dotonbori Delights

*   **Morning (9:00 AM - 1:00 PM): Explore Osaka Castle**
    Start your day with a visit to the iconic Osaka Castle, a significant historic landmark that dates back to 1583. Explore the castle's interior museum, which details its rich history and houses various artifacts, including samurai armor. Climb to the observation deck for panoramic views of the city. Don't forget to wander through the expansive Osaka Castle Park, a beautiful green oasis surrounding the castle. The castle is open daily from 9:00 AM to 5:00 PM, with the last admission at 4:30 PM, and it's recommended to allow 3 to 5 hours for a comprehensive visit, including the gardens.

*   **Lunch (1:00 PM - 2:00 PM): Casual Lunch near Osaka Castle or on the way to Dotonbori**
    Grab a quick and casual lunch near Osaka Castle, or head towards your next destination, Dotonbori, where you'll find plenty of dining options.

*   **Afternoon/Evening (2:00 PM onwards): Dotonbori Food and Atmosphere**
    Immerse yourself in the vibrant atmosphere of Dotonbori, Osaka's famous entertainment and food district. This area is known as "Japan's Kitchen" and is bustling with street food stalls and restaurants.
    *   **Food Exploration:** Sample local specialties such as takoyaki (octopus balls), kushikatsu (deep-fried skewers), and okonomiyaki (savory pancakes). Some highly recommended places for these dishes include Kushikatsu Daruma for kushikatsu and Chibo Okonomiyaki for okonomiyaki. You'll also find various other treats like ramen, gyoza, and crab dishes.
    *   **Sightseeing:** Enjoy the dazzling neon lights, the iconic Glico Running Man sign, and the lively atmosphere along the Dotonbori River.

How does this sound for your first day? Shall we plan Day 2 next?

--------------------------------------------------


Created a BRAND NEW session for Turn 2: 8b2e464a-ef3f-4170-9951-be0a96ff684f
🗣️ User (Turn 2): 'Yes, that looks perfect! Please plan Day 2.'

🚀 Running query for agent: 'multi_day_trip_agent' in session: '8b2e464a-ef3f-4170-9951-be0a96ff684f'...
EVENT: content=Content(
  parts=[
    Part(
      text="""Great! I'm glad Day 1 looks good. Let's move on to Day 2.

Here’s a suggestion for Day 2:

### Day 2: Cultural Immersion & Local Flavors

*   **Morning (9:00 AM - 1:00 PM): Explore a Major Museum or Historical Landmark**
    Start your day with a deep dive into the local culture or history. Visit a prominent art museum, a significant historical site, or a unique exhibition. This provides a rich educational experience and a chance to admire important collections or architecture.
*   **Lunch (1:00 PM - 2:30 PM): Authentic Local Cuisine**
    Enjoy lunch at a highly-rated local restaurant known for serving traditional dishes. Seek out a s

Great! I'm glad Day 1 looks good. Let's move on to Day 2.

Here’s a suggestion for Day 2:

### Day 2: Cultural Immersion & Local Flavors

*   **Morning (9:00 AM - 1:00 PM): Explore a Major Museum or Historical Landmark**
    Start your day with a deep dive into the local culture or history. Visit a prominent art museum, a significant historical site, or a unique exhibition. This provides a rich educational experience and a chance to admire important collections or architecture.
*   **Lunch (1:00 PM - 2:30 PM): Authentic Local Cuisine**
    Enjoy lunch at a highly-rated local restaurant known for serving traditional dishes. Seek out a spot popular with residents to get a true taste of the area's culinary scene.
*   **Afternoon (2:30 PM - 6:00 PM): Neighborhood Stroll & Unique Shopping**
    Spend your afternoon exploring one of the city's most charming or distinct neighborhoods. Wander through its streets, browse unique boutiques, artisan shops, or local markets. This is a great opportunity to find souvenirs and observe daily life.
*   **Evening (7:00 PM onwards): Live Performance or Dinner with a View**
    For your evening, consider catching a live performance, such as a play, concert, or local dance show, depending on what's available and your interests. Alternatively, enjoy a memorable dinner at a restaurant offering fantastic city views or a unique dining experience.

How does this sound for your second day?

--------------------------------------------------



See? The agent was confused! It likely asked what destination or what trip we were talking about. Because the second query was in a fresh, isolated session, the agent had no memory of planning Day 1 in Lisbon.

This perfectly illustrates why **managing sessions is the key to building truly conversational agents!**

---
## 🎉 Congratulations! 🎉

Congratulations on completing your ADK adventure into Tools and Memory! You've taken a massive leap from building single-shot agents to creating dynamic, stateful AI systems.

Let's recap the powerful concepts you've mastered:

- **Fundamental Agent & Tools**: You started by building a "Day Trip Genie" and equipped it with its first tool, GoogleSearch.

- **Custom Function Tools**: You gave your agent a new sense by creating a custom tool to fetch live data from the U.S. National Weather Service API.

- **Agent-as-a-Tool**: You orchestrated a sophisticated hierarchy where agents delegate tasks to other, more specialized agents, creating a collaborative team.

- **The Power of Memory**: Most importantly, you saw firsthand how managing a single, persistent Session allows an agent to remember context, adapt to user feedback, and conduct a meaningful, multi-turn conversation.

```
   __            /\_/\         /\_/\        /\_/\         __             (\__/)
o-''|\_____/).  ( o.o )       ( -.- )      ( ^_^ )     o-''|\_____/).    ( ^_^ )
 \_/|_)     )    > ^ <         > * <        >💖<         \_/|_)     )     / >🌸< \
    \  __  /                                              \  __  /         /   \
    (_/ (_/                                               (_/ (_/        (___|___)
```
